In [10]:
import streamlit as st
from streamlit_chat import message
from langchain.chains import ConversationalRetrievalChain
from langchain.embeddings import HuggingFaceEmbeddings,HuggingFaceInstructEmbeddings
from langchain.llms import LlamaCpp
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.memory import ConversationBufferMemory
from langchain.document_loaders import PyPDFLoader
import os
import tempfile

In [6]:
# Create llm
llm = LlamaCpp(
streaming = True,
model_path="D:\\Projects\\mini project\\LLM_Project - Copy\\Mistral-7B-Instruct-v0.1-GGUF\\mistral-7b-instruct-v0.1.Q4_K_M.gguf",
temperature=0.75,
top_p=1, 
verbose=True,
n_ctx=4096
)


llama_model_loader: loaded meta data with 20 key-value pairs and 291 tensors from D:\Projects\mini project\LLM_Project - Copy\Mistral-7B-Instruct-v0.1-GGUF\mistral-7b-instruct-v0.1.Q4_K_M.gguf (version GGUF V2)
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = mistralai_mistral-7b-instruct-v0.1
llama_model_loader: - kv   2:                       llama.context_length u32              = 32768
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_count u32              = 32
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 14336
llama_model_loader: - kv   6:                 llama.rope.dimension_count u32  

In [7]:
# embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2", 
#                                            model_kwargs={'device': 'cpu'})

In [11]:
embeddings = HuggingFaceInstructEmbeddings(
    model_name = os.path.join(os.getcwd(), "instruct-large"),
    model_kwargs = {"device" : "cpu"},
    encode_kwargs = {"normalize_embeddings":True},
    query_instructions = "Represent the query for retrieval: "
)

ValidationError: 1 validation error for HuggingFaceInstructEmbeddings
query_instructions
  extra fields not permitted (type=value_error.extra)

In [ ]:
# def DB_recreation():
#     try:
#         if os.path.exists(os.path.join(os.getcwd(), "faiss_index.pkl")):
#             os.remove(os.path.join(os.getcwd(), "faiss_index.pkl"))

#         uploaded_file_path = ""

#         files = os.listdir(uploaded_file_path)

#         for index,file in enumertae(files):
#             try:
#                 filename,_ = os.path.splitext(file)
#                 if (file.split('.'[1]) == 'pdf':
#                     loader = PyPDFLoader(uploaded_file_path+file)
#                     docs1 = loader.load_and_split()
#                 else:
#                     loader = Textoader(upload_file_path+file)
#                     docs1 = loader.load()
#                     text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=32)
#                     docs1 = text_splitter.split_documents(docs1)
    
#                 for count in range(len(docs1)):
#                     docs1[count].metadata["filename"] = filename
#                 if index == 0:
#                     db = FAISS.from_documents(docs1, embeddings)
#                     db.merge_from(db_new)

#             except Exception as e:
#                 print(f"error in files: ",e)

#             pkl = db.serialize_to_bytes()

#             with open('faiss_index.pkl', 'wb') as file:
#                 pickle.dump(pkl, file)

            

In [12]:
temp_file_path = ""
loader = PyPDFLoader(temp_file_path)
data = loader.load()

ImportError: pypdf package not found, please install it with `pip install pypdf`

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=10000, chunk_overlap=20)
text_chunks = text_splitter.split_documents(data)

In [ ]:
vector_store = FAISS.from_documents(text_chunks, embedding=embeddings)

In [ ]:
# chain = create_conversational_chain(vector_store)
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

In [ ]:
chain = ConversationalRetrievalChain.from_llm(llm=llm, chain_type='stuff',
                                             retriever=vector_store.as_retriever(search_kwargs={"k": 2}),
                                             memory=memory)
    

In [4]:
pip install llama-cpp-python

  Using cached llama_cpp_python-0.2.56-cp311-cp311-win_amd64.whl
  Using cached diskcache-5.6.3-py3-none-any.whl.metadata (20 kB)
Using cached diskcache-5.6.3-py3-none-any.whl (45 kB)
Note: you may need to restart the kernel to use updated packages.
